# 5. Baseline EMA Strategy with Regime Filter
## 5/15 EMA Crossover Strategy Implementation and Backtesting

In [1]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_utils import load_data
from strategy import EMAStrategy, TradeAnalyzer
from backtest import Backtester, BacktestVisualizer, generate_backtest_report, save_backtest_results
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [6]:
# Load merged data with regimes from previous notebook
df = load_data('../data/nifty_merged_5min.csv')
df['returns'] = df['close_spot'].pct_change()

# Fit simple regime detector for this notebook
from regime import RegimeDetector
detector = RegimeDetector(n_regimes=3)
train_size_regime = int(0.7 * len(df))
detector.fit(df.iloc[:train_size_regime], feature_cols=['returns'])
df['regime'] = detector.predict(df)

print(f"Data shape: {df.shape}")
print(f"Regime distribution: {df['regime'].value_counts().to_dict()}")
df.head()

INFO:data_utils:Loading data from ../data/nifty_merged_5min.csv
INFO:regime:Initialized RegimeDetector with 3 regimes
INFO:regime:Fitting HMM to training data
INFO:regime:Preparing 1 features for HMM
INFO:regime:Features prepared and normalized
INFO:regime:HMM fitted successfully. Log-likelihood: -220.79
INFO:regime:Predicting regimes
INFO:regime:Preparing 1 features for HMM
INFO:regime:Features prepared and normalized


Data shape: (245, 18)
Regime distribution: {0: 122, 1: 122, 2: 1}


,timestamp,open_spot,high_spot,low_spot,close_spot,volume_spot,open_futures,high_futures,low_futures,close_futures,volume_futures,open,high,low,close,volume,returns,regime
0,2025-01-20,23344.750000,23391.099609,23170.650391,23290.400391,301500,49350.800781,49650.601562,48683.601562,48834.148438,173100,1761.179077,1775.407887,1741.608239,1770.551605,4333011,NaN,0
1,2025-01-21,23024.650391,23426.300781,22976.849609,23421.650391,312900,48570.898438,49543.148438,48430.949219,49532.000000,134700,1748.941162,1779.001616,1741.511146,1766.763810,7169109,0.005635,1
2,2025-01-22,23155.349609,23169.550781,22981.300781,23099.150391,276000,48724.398438,48781.750000,48074.050781,48689.550781,151700,1803.088745,1812.170087,1753.263367,1755.448694,8373913,-0.013769,0
3,2025-01-23,23205.349609,23270.800781,23090.650391,23128.300781,275600,48589.000000,48892.699219,48493.000000,48770.148438,124600,1811.829956,1825.524764,1799.446439,1804.594144,5907471,0.001262,1
4,2025-01-24,23092.199219,23347.300781,23050.000000,23183.900391,264300,48367.800781,48858.648438,48203.000000,48546.050781,139900,1821.542603,1840.433609,1810.664579,1811.878650,4887027,0.002404,0


## 5.1 Split Data (Training: 70%, Testing: 30%)

In [17]:
# Split data
backtester = Backtester(initial_capital=100000, commission=0.0003)
train_df, test_df = backtester.split_data(df, train_size=0.7)

print(f"Training set: {len(train_df)} rows")
print(f"Testing set: {len(test_df)} rows")

INFO:backtest:Initialized Backtester with capital=100000, commission=0.0003
INFO:backtest:Data split: Train=171, Test=74


Training set: 171 rows
Testing set: 74 rows


## 5.2 Implement EMA Strategy

In [21]:
import importlib
import backtest as backtest_module
importlib.reload(backtest_module)
from backtest import Backtester
backtester = Backtester()

train_df['ema_5'] = train_df['close_spot'].ewm(span=5).mean()
train_df['ema_15'] = train_df['close_spot'].ewm(span=15).mean()
test_df['ema_5'] = test_df['close_spot'].ewm(span=5).mean()
test_df['ema_15'] = test_df['close_spot'].ewm(span=15).mean()

strategy = EMAStrategy(fast_ema=5, slow_ema=15)
train_df = strategy.generate_signals(train_df, use_regime_filter=True)
test_df = strategy.generate_signals(test_df, use_regime_filter=True)

train_df = strategy.generate_positions(train_df)
test_df = strategy.generate_positions(test_df)

train_df = backtester.calculate_returns(train_df)
test_df = backtester.calculate_returns(test_df)

# Set timestamp as index for metric calculation
train_df_indexed = train_df.set_index('timestamp')
test_df_indexed = test_df.set_index('timestamp')

train_results = {'metrics': backtester.calculate_metrics(train_df_indexed)}
test_results = {'metrics': backtester.calculate_metrics(test_df_indexed)}

print("\n✓ Backtest completed!")
print(f"  Train Return: {train_results['metrics']['total_return']:.2f}%")
print(f"  Test Return: {test_results['metrics']['total_return']:.2f}%")

INFO:backtest:Initialized Backtester with capital=100000, commission=0.0003
INFO:strategy:Initialized EMA Strategy: Fast=5, Slow=15
INFO:strategy:Generating trading signals
INFO:strategy:Signals generated with regime filter
INFO:strategy:Generating trading signals
INFO:strategy:Signals generated with regime filter
INFO:strategy:Generating positions from signals
INFO:strategy:Positions generated. Long: 82, Short: 0, Flat: 89
INFO:strategy:Generating positions from signals
INFO:strategy:Positions generated. Long: 11, Short: 0, Flat: 63
INFO:backtest:Calculating strategy returns
INFO:backtest:Returns calculated
INFO:backtest:Calculating strategy returns
INFO:backtest:Returns calculated
INFO:backtest:Calculating performance metrics
INFO:strategy:Extracting individual trades
INFO:strategy:Extracted 7 trades
INFO:strategy:Calculating trade statistics
INFO:strategy:Trade statistics calculated. Win rate: 28.57%
INFO:backtest:Metrics calculated. Sharpe: -0.52, Max DD: -3.85%
INFO:backtest:Calcu


✓ Backtest completed!
  Train Return: 1.16%
  Test Return: -1.45%


## 5.3 Backtest on Training Set

In [ ]:
# Run backtest on training data
train_results = backtester.backtest(train_df, strategy_name="EMA 5/15 with Regime Filter (Train)")

print("\nTraining Set Performance:")
print("=" * 80)
metrics = train_results['metrics']
print(f"Total Return: {metrics['total_return']:.2f}%")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Sortino Ratio: {metrics['sortino_ratio']:.2f}")
print(f"Calmar Ratio: {metrics['calmar_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2f}%")
print(f"Win Rate: {metrics['win_rate']:.2f}%")
print(f"Profit Factor: {metrics['profit_factor']:.2f}")
print(f"Total Trades: {metrics['total_trades']}")

## 5.4 Backtest on Testing Set

In [ ]:
# Run backtest on testing data
test_results = backtester.backtest(test_df, strategy_name="EMA 5/15 with Regime Filter (Test)")

print("\nTesting Set Performance:")
print("=" * 80)
metrics = test_results['metrics']
print(f"Total Return: {metrics['total_return']:.2f}%")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Sortino Ratio: {metrics['sortino_ratio']:.2f}")
print(f"Calmar Ratio: {metrics['calmar_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2f}%")
print(f"Win Rate: {metrics['win_rate']:.2f}%")
print(f"Profit Factor: {metrics['profit_factor']:.2f}")
print(f"Total Trades: {metrics['total_trades']}")

## 5.5 Visualize Results

In [ ]:
# Calculate returns for visualization
test_df_with_returns = backtester.calculate_returns(test_df)

# Plot equity curve
BacktestVisualizer.plot_equity_curve(
    test_df_with_returns,
    save_path='../plots/baseline_equity_curve.png'
)
print("Equity curve plot saved")

In [ ]:
# Plot drawdown
BacktestVisualizer.plot_drawdown(
    test_df_with_returns,
    save_path='../plots/baseline_drawdown.png'
)
print("Drawdown plot saved")

In [ ]:
# Plot returns distribution
BacktestVisualizer.plot_returns_distribution(
    test_df_with_returns,
    save_path='../plots/baseline_returns_distribution.png'
)
print("Returns distribution plot saved")

In [ ]:
# Plot trade analysis
if test_results['trades'] is not None and len(test_results['trades']) > 0:
    BacktestVisualizer.plot_trade_analysis(
        test_results['trades'],
        save_path='../plots/baseline_trade_analysis.png'
    )
    print("Trade analysis plot saved")

## 5.6 Generate Reports

In [ ]:
# Generate backtest report
generate_backtest_report(
    test_results,
    output_path='../results/baseline_strategy_report.txt'
)
print("Backtest report generated")

# Display report
with open('../results/baseline_strategy_report.txt', 'r') as f:
    print(f.read())

In [ ]:
# Save detailed results
save_backtest_results(test_results, '../results/baseline_strategy')
print("Detailed results saved to results/baseline_strategy_*")

## Summary

In [22]:
print("=" * 80)
print("BASELINE STRATEGY SUMMARY")
print("=" * 80)
print(f"\nStrategy: 5/15 EMA Crossover with Regime Filter")
print(f"Training Period: {len(train_df)} candles (70%)")
print(f"Testing Period: {len(test_df)} candles (30%)")
print(f"\nTest Set Results:")
print(f"  Total Return: {test_results['metrics']['total_return']:.2f}%")
print(f"  Sharpe Ratio: {test_results['metrics']['sharpe_ratio']:.2f}")
print(f"  Win Rate: {test_results['metrics']['win_rate']:.2f}%")
print(f"  Total Trades: {test_results['metrics']['total_trades']}")
print("\nNext Step: Proceed to 06_ml_models.ipynb")

BASELINE STRATEGY SUMMARY

Strategy: 5/15 EMA Crossover with Regime Filter
Training Period: 171 candles (70%)
Testing Period: 74 candles (30%)

Test Set Results:
  Total Return: -1.45%
  Sharpe Ratio: -3.72
  Win Rate: 0.00%
  Total Trades: 1

Next Step: Proceed to 06_ml_models.ipynb
